In [1]:
# !git clone https://github.com/GabrielGlzSa/ModelCompression.git
# %cd ModelCompression/
# !git checkout --track origin/CompressionV2
# !pip install Pillow \
#     h5py \
#     keras_preprocessing \
#     matplotlib \
#     mock \
#     numpy \
#     scipy \
#     sklearn \
#     pandas \
#     future \
#     portpicker \
#     enum34 \
#     tensorflow==2.6.2 \
#     tensorflow_datasets==4.0.1\
#     tensorflow_transform\
#     tensorboard_plugin_profile\
#     seaborn\
#     pyparsing==2.4.7\
#     tf_agents \
#     tensorflow-model-optimization
%load_ext tensorboard

In [2]:
# from google.colab import drive
# import sys

# drive.mount('/content/drive/')
# %cd ./drive/MyDrive/Colab Projects/
# %ls

sys.path.insert(0, './ModelCompression')
dataset = 'mnist'

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import logging
import pandas as pd
from IPython.display import clear_output
from utils import load_dataset
from CompressionTechniques import *
from replay_buffer import ReplayBuffer
from environments import *
from custom_layers import *
logging.basicConfig(level=logging.INFO, format='%(asctime)s -%(levelname)s - %(funcName)s -  %(message)s')

/home/A00806415/anaconda3/envs/mc/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# kernel_size = (3,3)
# channels = 3
# bases = 4
# filters = 32
# fh, fw = kernel_size
# S = np.random.random(size=(channels, bases, filters))
# Q = np.random.random(size=(channels, fh, fw, bases))
# P = np.random.random(size=(channels, channels))
# v1 = SparseConvolution2D(kernel_size=kernel_size,
#                            filters=filters,
#                            PQS=[P, Q, S],
#                            bases=bases)

# imgs = tf.constant(np.random.randint(0, 255, size=(4,32,32,3)).astype(np.float32))
# %timeit v1(imgs)

In [6]:
def make_env(dataset):
    train_ds, valid_ds, test_ds, input_shape, num_classes = load_dataset(dataset)

    model_path = './data/full_model/test_'+dataset
    optimizer = tf.keras.optimizers.Adam()
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
    train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    try:
      model = tf.keras.models.load_model(model_path, compile=True)
    except OSError:
      
      model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_0',
                                                          input_shape=input_shape),
                                   tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_1'),
                                   tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_2'),
                                   tf.keras.layers.MaxPool2D((2, 2), 2),
                                   tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(128, activation='relu', name='dense_0'),
                                   tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
                                   tf.keras.layers.Dense(num_classes, activation='softmax', name='dense_softmax')
                                   ])
      model.compile(optimizer=optimizer, loss=loss_object, metrics=train_metric)
      model.fit(train_ds, epochs=5, validation_data=valid_ds)
      model.save(model_path)

    parameters = {}
    parameters['DeepCompression'] = {'layer_name': 'dense_0', 'threshold': 0.001}
    parameters['ReplaceDenseWithGlobalAvgPool'] = {'layer_name': 'dense_1'}
    parameters['InsertDenseSVD'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSVDCustom'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSparse'] = {'layer_name': 'dense_0', 'verbose': True, 'units': 16}
    parameters['InsertSVDConv'] = {'layer_name': 'conv2d_1', 'units': 8}
    parameters['DepthwiseSeparableConvolution'] = {'layer_name': 'conv2d_1'}
    parameters['FireLayerCompression'] = {'layer_name': 'conv2d_1'}
    # parameters['MLPCompression'] = {'layer_name': 'conv2d_1'}
    parameters['SparseConnectionsCompression'] = {'layer_name': 'conv2d_1', 'epochs': 20,
                                                  'target_perc': 0.75, 'conn_perc_per_epoch': 0.1}
    # compressor = ReplaceDenseWithGlobalAvgPool(model=model, dataset=train_ds,
    #                                            optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer()


    # compressor = InsertDenseSVD(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric, fine_tune=False)
    # compressor.compress_layer('dense_1', units=32)


    # compressor = InsertDenseSparse(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('dense_1', verbose=True)


    # compressor = InsertSVDConv(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # compressor = DepthwiseSeparableConvolution(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # compressor = FireLayerCompression(model=model, dataset=train_ds,
    #                                   optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    compressor = MLPCompression(model=model, dataset=train_ds,
                                optimizer=optimizer, loss=loss_object, metrics=train_metric, input_shape=input_shape)
    compressor.compress_layer('conv2d_1')

    # population=20,generations=100 takes 3 hours and 40 minutes
    # compressor = SparseConnectionsCompressionCustom(model=model,
    #                                                 dataset=train_ds,
    #                                                 optimizer=optimizer,
    #                                                 loss=loss_object,
    #                                                 metrics=train_metric)
    # compressor.compress_layer('conv2d_1', epochs=10)

    # RAM OOM in training loop.
    # compressor = SparseConvolutionCompression(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric, bases=4, input_shape=input_shape)
    # compressor.compress_layer('conv2d_1', iterations=2)

    new_model = compressor.get_model()
    new_model.summary()


make_env(dataset)

2022-05-16 14:05:51,140 -INFO - read_from_directory -  Load dataset info from ./data/mnist/3.0.1
2022-05-16 14:05:51,144 -INFO - download_and_prepare -  Reusing dataset mnist (./data/mnist/3.0.1)
2022-05-16 14:05:51,146 -INFO - as_dataset -  Constructing tf.data.Dataset for split ['train[:80%]', 'train[80%:]', 'test'], from ./data/mnist/3.0.1
2022-05-16 14:05:51,632 -INFO - compress_layer -  Searching for layer: conv2d_1
2022-05-16 14:05:51,641 -INFO - compress_layer -  Learning MLP filter.
2022-05-16 14:06:55,081 -INFO - fine_tune_model -  Fine-tuning the whole optimized model.


Epoch 1/2
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0161 - sparse_categorical_accuracy: 0.9948
Epoch 2/2
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0123 - sparse_categorical_accuracy: 0.9963


2022-05-16 14:07:07,404 -INFO - compress_layer -  Replaced layer was using 9248 weights.
2022-05-16 14:07:07,406 -INFO - compress_layer -  MLPConv is using 5168 weights.
2022-05-16 14:07:07,407 -INFO - compress_layer -  Finished compression


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_0 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1/MLPConv (MLPConv)   (None, 24, 24, 32)        5168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0         
_________________________________________________________________
dense_0 (Dense)              (None, 128)               4957

: 